In [5]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from sklearn.model_selection import train_test_split  
import numpy as np
import matplotlib.pyplot as plt
import pickle

print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print(f"Numpy version: {np.__version__}")

PyTorch version: 2.5.1
Torchvision version: 0.20.1
Numpy version: 1.26.4


In [ ]:
## Importation des données puis formatage pour l'entrainement

#load data from data.pkl in a tensor
with open('data.pkl', 'rb') as file: #récupération des données
    data = pickle.load(file)
    data = torch.tensor(data)

# Mélanger les lignes  
permuted_indices = torch.randperm(data.size(0))  # Générer des indices aléatoires  
shuffled_data = data[permuted_indices]  # Réorganiser les données  

# Diviser le tenseur en ensembles d'entraînement, de validation et de test  
train_size = int(0.8 * len(shuffled_data))  
val_size = int(0.1 * len(shuffled_data))  
test_size = len(shuffled_data) - train_size - val_size  

train_data = shuffled_data[:train_size]  # Ensemble d'entraînement  
val_data = shuffled_data[train_size:train_size + val_size]  # Ensemble de validation  
test_data = shuffled_data[train_size + val_size:]  # Ensemble de test  


print("Ensemble d'entraînement :", train_data.shape)  
print("Ensemble de validation :", val_data.shape)  
print("Ensemble de test :", test_data.shape)

Ensemble d'entraînement : torch.Size([400, 4098])
Ensemble de validation : torch.Size([50, 4098])
Ensemble de test : torch.Size([50, 4098])
